In [2]:
import random
import numpy as np

In [3]:
class Vehicle:
    # This class is used to represent vehicles
    """
        :param battery_capacity(KWh): double
            description: Maximum capacity of vehicle battery
        :param start_time: datetime
            description: The time the owner began using the vehicle
        :param end_time: datetime
            description: The time when the vehicle owner finishes using the vehicle
        :param parking_lot: object
            description: The parking lot where the vehicle is parked
        :param battery_level(KWh): double
            description: The current available electricity consumption of the vehicle
        :param charging_power(KW): int
            description: The charge and discharge power of the vehicle
    """
    def __init__(self, start_time, end_time, parking_lot=None, battery_capacity=77.0, battery_level=77.0, charging_power=20):
        self.start_time = start_time
        self.end_time = end_time
        self.battery_capacity = battery_capacity
        self.battery_level = battery_level
        self.charging_power = charging_power

    #充电一小时
    def charge(self, price):
        temp = self.battery_level
        if self.battery_level <= self.battery_capacity - 20:
            self.battery_level += 20
        else:
            self.battery_level = self.battery_capacity
        return 0 - (self.battery_level - temp) * price
    
    #放电一小时
    def discharge(self,price):
        temp = self.battery_level
        if self.battery_level >= 20:
            self.battery_level -= 20
        else:
            self.battery_level = 0
        return 0 - (self.battery_level - temp) * price
    
    def __str__(self):
        return f'Vehicle: {self.battery_capacity}, {self.start_time}, {self.end_time}'
    

In [4]:
from datetime import datetime, timedelta

def generate_random_time(start_hour, end_hour):
    hour = random.randint(start_hour, end_hour)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    return datetime.now().replace(hour=hour, minute=minute, second=second)

# # 生成第一个时间，范围在七点到九点
# start_time = generate_random_time(7, 9)

# # 生成第二个时间，范围在四点到六点
# end_time = generate_random_time(16, 18)

# print("随机生成的第一个时间:", start_time.strftime('%H:%M:%S'))
# print("随机生成的第二个时间:", end_time.strftime('%H:%M:%S'))


In [5]:
#根据生成的到达时间和离开时间计算出实际情况下的开始充电和停止充电时间(整点)
def calculate_real_time(start_time, end_time):
    real_start_time = (start_time + timedelta(hours = 1))
    real_start_time = real_start_time.replace(minute=0, second=0)
    real_end_time = (end_time - timedelta(hours = 0))
    real_end_time = real_end_time.replace(minute=0, second=0)

    return real_start_time, real_end_time
    

In [6]:
import pandas as pd

#读取一天（即24小时）的电价
def read_e_price(start, end):
    df_prices = pd.read_csv("GR-data-11-20.csv", header=None, names=["DateTime", "ElectricityPrice"])
    one_day_prices_list = df_prices["ElectricityPrice"][start:end].tolist()

    return one_day_prices_list


In [7]:
#初始化读取电价的索引
start_index = 0
end_index = 24

#初始化奖励
reward = 0

# 生成上班时间，范围在七点到九点
start_time = generate_random_time(7, 9)
# 生成下班时间，范围在四点到六点
end_time = generate_random_time(16, 18)
#计算实际开始充电和结束充电的时间（整点）
real_start_time, real_end_time = calculate_real_time(start_time, end_time)

#初始化环境
env = Vehicle(start_time, end_time)
env.__str__()

for i in range(7):
    #读入一天的电价，并更新start_index和end_index
    one_day_prices_list = read_e_price(start_index, end_index)
    start_index += 24
    end_index += 24

    current_time = real_start_time

    # 模拟一天内时间的变化，每次增加60分钟
    for _ in range(real_end_time.hour - real_start_time.hour):  

        # 将时间增加60分钟
        current_time += timedelta(minutes=60)

        random_number = random.randint(0, 1)
        if random_number == 1:
            reward += env.charge(one_day_prices_list[current_time.hour] / 1000)
        else:
            reward += env.discharge(one_day_prices_list[current_time.hour] / 1000)

        if current_time.hour == real_start_time.hour or current_time.hour == real_end_time.hour:
            if env.battery_level < 10:
                # reward -= 500
                print("SOC is less than 10%: ", env.battery_level)

    # 更新第二天上下班时间
    start_time = generate_random_time(7, 9)
    end_time = generate_random_time(16, 18)
    env.start_time = start_time
    env.end_time = end_time

    #计算实际开始充电和结束充电的时间（整点）
    real_start_time, real_end_time = calculate_real_time(env.start_time, env.end_time)

print("Total_Reward", round(reward,2))
    

Total_Reward 0.47
